In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

C:\Users\TonyLab\PycharmProjects\MemeGAN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. LLM Test

In [5]:
### 不確定是否為官方的 Gemini ############################################################################
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")
gemini = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")
#######################################################################################################
gemini

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [15]:
### 官方的Gemma #########################################################################################
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", device_map="auto",  torch_dtype=torch.bfloat16)
########################################################################################################
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[2, 1, 3, 0, 106, 107]
['<bos>', '<eos>', '<unk>', '<pad>', '<start_of_turn>', '<end_of_turn>']


In [2]:
### 官方的Gemma (Instruction-tuning)#####################################################################
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", device_map="auto",  torch_dtype=torch.bfloat16)
########################################################################################################
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.76s/it]


[2, 1, 3, 0, 106, 107]
['<bos>', '<eos>', '<unk>', '<pad>', '<start_of_turn>', '<end_of_turn>']


In [17]:
a= torch.tensor([[[0.100,0.2], [0.2000 ,0.300], [0.300, 0.300]],[[0.100,0.2], [0.2000 ,0.300], [0.300, 0.300]]]).to(torch.bfloat16)
a.shape

torch.Size([2, 3, 2])

In [19]:
#mean dim -1
print(torch.mean(a, dim=-2).shape)
torch.mean(a, dim=-2)

torch.Size([2, 2])


tensor([[0.2002, 0.2676],
        [0.2002, 0.2676]], dtype=torch.bfloat16)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Complete sentence with following elements : apple, nice, cool, amaze, shiny, device, format, pants, rainy, smooth, compute, ear, card, normal, pacific. "
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding='max_length', max_length=64).to(device)
# Complete會出填空題給我
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><bos>Complete sentence with following elements : apple, nice, cool, amaze, shiny, device, format, pants, rainy, smooth, compute, ear, card, normal, pacific. 

**The cool apple device, with its shiny surface and smooth compute format, amaze me. I'm wearing my normal pants, and it's a rainy day in the Pacific.** 

Let me know if you'd like to try another sentence! 
<end_of_turn>


# Complete normal 會出填空題給我

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Complete sentence with following elements : apple, nice, cool, amaze. "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# Complete會出填空題給我
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Complete sentence with following elements : apple, nice, cool, amaze. 1. I was really _________________ when I saw the new iPhone. 2. The new iPhone is really _________________. 3. The new iPhone is really _________________. 4. The new iPhone is really _________________. 5. The new iPhone is really _________________. 6. The new iPhone is really _________________. 7. The new iPhone is really _________________. 8. The new iPhone is really _________________. 9. The new iPhone is really _________________. 10. The new iPhone is really _________________.

Complete sentence with following elements : apple, nice, cool, amaze. 1. I was really _________________ when I saw the new iPhone. 2. The new iPhone is really _________________. 3. The new iPhone is really _________________. 4. The new iPhone is really _________________. 5. The new iPhone is really _________________. 6. The new iPhone is really ________________


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Complete sentence with following elements : apple, nice, cool, amaze. "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# Complete會出填空題給我
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\models\gemma2\modeling_gemma2.py:458: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<bos>Complete sentence with following elements : apple, nice, cool, amaze. 

The apple was **nice** and **cool** to the touch. It **amazed** me with its smooth, shiny skin. 


Let me know if you'd like to try another sentence! 
<end_of_turn>


# involve normal會開始比賽

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Generate a funny caption involve following words: apple, nice, cool, amaze. "
# 用involve會開始比賽
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Generate a funny caption involve following words: apple, nice, cool, amaze. 

The best caption will be chosen by the author of the post.

The winner will be announced on the 1st of December.

The winner will receive a 1000 coins.

The winner will be chosen by the author of the post.

The winner will be announced on the 1st of December.

The winner will receive a 1000 coins.

The winner will be chosen by the author of the post.

The winner will be announced on the 1st of December.

The winner will receive a 1000 coins.

The winner will be chosen by the author of the post.

The winner will be announced on the 1st of December.

The winner will receive a 1000 coins.

The winner will be chosen by the author of the post.

The winner will be announced on the 1st of December.

The winner will receive a 1000 coins


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Generate a funny caption involve following words: apple, nice, cool, amaze. "
# 用involve會開始比賽
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Generate a funny caption involve following words: apple, nice, cool, amaze. 

Here are some ideas:

**Option 1:**

"This apple is so nice, it's cool to the touch and amazes me with its perfect shape. 🍎"

**Option 2:**

"This apple is so amazeballs, it's nice enough to eat and cool enough to be a conversation starter. 🍎"

**Option 3:**

"I'm not sure what's more amazeballs, this apple or the fact that I found it in my fridge. 🍎"


Choose the best option for you and add your own personal touch! 
<end_of_turn>


# elements normal會拆解詞性

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Create a funny sentence that contains following elements: apple, nice, cool, amaze. "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Create a funny sentence that contains following elements: apple, nice, cool, amaze. 

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb
* an adjective
* an adverb

Write a sentence that contains the following elements:

* a noun
* a verb


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Create a funny sentence that contains following elements: apple, nice, cool, amaze. "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Create a funny sentence that contains following elements: apple, nice, cool, amaze. 

An apple a day keeps the doctor away, but a nice, cool, amaze-ing apple a day keeps the doctor away from your house! 
<end_of_turn>


# elements normal會拆解詞性

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write one Post for Instagram possibly with emoji that contains following words: apple, nice, cool, amaze. Answer:"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write one Post for Instagram possibly with emoji that contains following words: apple, nice, cool, amaze.Answer: Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write one Post for Instagram possibly with emoji that contains following words: apple, nice, cool, amaze. Answer:"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write one Post for Instagram possibly with emoji that contains following words: apple, nice, cool, amaze. Answer:

**What's your favorite thing about fall? 🍂🍎  I love the crisp air, the colorful leaves, and the delicious apple cider! 🍎  It's just so nice and cool, and it amazes me how much nature can change in just a few weeks! 🍁  What's your favorite thing about fall? 👇**

**Explanation:**

* **Engaging Question:** The post starts with a question to encourage interaction and get people talking.
* **Relevant Words:** The post uses the words "apple," "nice," "cool," and "amaze" to connect with the theme of fall.
* **Descriptive Language:** The post uses descriptive language to paint a picture of fall, like "crisp air," "colorful leaves," and "delicious apple cider."
* **Call to Action:** The post ends with a call to action, asking people to share their favorite things about fall.


Let me know if you'd like


# one normal 會長篇大論

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write one short Post for Instagram possibly with emoji that contains all of the following words: apple, nice, cool, amaze. Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write one short Post for Instagram possibly with emoji that contains all of the following words: apple, nice, cool, amaze. Answer: 1. Apple is a fruit that is very popular in the world. It is a fruit that is very delicious and has a lot of benefits for the body. 2. Nice is a word that means good. It is a word that is used to describe something that is good or nice. 3. Cool is a word that means cool. It is a word that is used to describe something that is cool or cool. 4. Amaze is a word that means amaze. It is a word that is used to describe something that is amaze or amaze.<eos>


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write one short Post for Instagram possibly with emoji that contains all of the following words: apple, nice, cool, amaze. Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write one short Post for Instagram possibly with emoji that contains all of the following words: apple, nice, cool, amaze. Answer: 

**Apple, nice, cool, amaze**

**Post:** 
Just had the coolest apple I've ever tasted! 🍎 It was so nice and sweet, it totally amaze me! 🤩 #applelove #foodie #sweettooth 

**Explanation:**

* **Apple:**  The core subject of the post.
* **Nice:**  Describes the taste of the apple.
* **Cool:**  Highlights the coolness of the apple.
* **Amazes:**  Expresses the feeling of being impressed by the apple's taste. 


Let me know if you'd like to try another one! 😊 
<end_of_turn>


# two normal會重複

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze. Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze. Answer: 🍎 🍏 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze. Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze. Answer: 

**"Feeling **nice** and **cool** today? 🍎  Let's **amaze** ourselves with a delicious apple! 🍏"

**Here are some ideas for your Instagram caption:**

* **Playful:** "Feeling **nice** and **cool** today? 🍎  Let's **amaze** ourselves with a delicious apple! 🍏  What's your favorite way to eat an apple?"
* **Motivational:** "**Nice** and **cool** vibes only! 😎  Let's **amaze** ourselves with a fresh apple today. 🍎  What are your goals for the week?"
* **Simple:** "**Nice** and **cool** day. ☀️  **Amaze** yourself with a delicious apple! 🍎"


**Which one do you like best?** 
<end_of_turn>


# emoji normal 不會出來 ; it + \<pad\> 會 \<eos\>

In [125]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emojis and blue, hate, sleep, monday. Answer:<pad>"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emojis and blue, hate, sleep, monday. Answer:<pad>.

<h3>The best way to get a good night’s sleep is to go to bed at the same time every night and get up at the same time every morning.</h3>

<strong>Sleep emoji copy and paste</strong>. Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy and paste. Sleep emoji copy and paste. Sleep emoji copy and paste.

Sleep emoji copy


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emojis and blue, hate, sleep, monday. Answer:<pad>"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emojis and blue, hate, sleep, monday. Answer:<pad><eos>


# 換行 Answer normal 比較不會重複生成

In [148]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Write a copy for Instagram with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Write a copy for Instagram with all of the following words: blue, hate, sleep, monday.
 Answer:
Blue Monday is the most depressing day of the year. It’s the day after New Year’s Day, when many people are still trying to stick to their resolutions and are feeling down about their progress. This day is also known as “Blue Monday” because it’s the day when people are most likely to feel blue.
The day after New Year’s Day is a day when people are most likely to feel blue. This is because they are still trying to stick to their resolutions and are feeling down about their progress. This day is also known as “Blue Monday” because it’s the day when people are most likely to feel blue.
The day after New Year’s Day is a day when people are most likely to feel blue. This is because they are still trying to stick to their resolutions and are feeling down about their progress. This day is also known as “Blue Monday” because it’s the day when people are most likely to feel blue.


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Write a copy for Instagram with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Write a copy for Instagram with all of the following words: blue, hate, sleep, monday.
 Answer: 

**Blue Monday blues got you down? 😩  Hate Mondays?  😴  We feel you!  Let's get through this week together. 💪  What are your favorite ways to beat the Monday morning slump?  ⬇️**

**#MondayMotivation #BlueMonday #Sleep #Motivation**


**Explanation:**

* **Catchy opening:**  "Blue Monday blues got you down?" immediately connects with the target audience and their feelings.
* **Relatable emotions:**  "Hate Mondays?" and "😴" acknowledge the common struggle and make the post more personal.
* **Positive call to action:**  "Let's get through this week together" encourages engagement and community.
* **Open-ended question:**  "What are your favorite ways to beat the Monday morning slump?" invites followers to share their tips and connect.
* **Relevant hashtags:**  #MondayMotivation, #BlueMonday, #Sleep, and #Motivation are all relevant to the


# 加Prompt Answer

In [156]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate one simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate one simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.
 Answer:
Blue Monday is a day of the week that is often associated with feelings of sadness, depression, and anxiety. It is the third Monday of January, and it is often considered to be the most depressing day of the year. This is because it is the day after New Year’s Day, which is often a time of celebration and happiness. However, for many people, the reality of the day is much different. They may feel overwhelmed by the responsibilities of the new year, or they may be struggling with the loss of a loved one. Whatever the reason, Blue Monday can be a difficult day to get through.
If you are feeling blue on Blue Monday, there are a few things you can do to help yourself. First, try to get outside and get some fresh air. This can help to clear your head and make you feel better. Second, try to focus on the positive things in your life. This can help to s

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate one simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate one simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.
 Answer: 😴💙 Monday blues got me hating life. 
<end_of_turn>


# 加Prompt Answer +

In [159]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate two simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate two simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.
 Answer:
1. Blue Monday is a day of the week that is often associated with feelings of sadness and depression.
2. Hate Monday is a day that is often associated with feelings of anger and resentment.
3. Sleep Monday is a day that is often associated with feelings of fatigue and exhaustion.
4. Monday is a day that is often associated with feelings of dread and anxiety.
5. Blue Monday is a day that is often associated with feelings of sadness and depression.
6. Hate Monday is a day that is often associated with feelings of anger and resentment.
7. Sleep Monday is a day that is often associated with feelings of fatigue and exhaustion.
8. Monday is a day that is often associated with feelings of dread and anxiety.
9. Blue Monday is a day that is often associated with feelings of sadness and depression.
10. Hate Monday is a day that is often associated with fee

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate two simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate two simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.
 Answer:

1.  **Blue** skies and a **Monday** morning feeling. 😴  #MondayMotivation 
2.  **Hate** Mondays, but **sleep** is the only escape. 😴 #MondayBlues 

Let me know if you'd like to try another prompt! 😊 
<end_of_turn>


# 加Prompt Answer + 

In [171]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer:
1. Pink hate sleep Monday
2. Pink hate sleep Monday
3. Pink hate sleep Monday
Explanation:
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second sentence is "Pink hate sleep Monday." The third sentence is "Pink hate sleep Monday."
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second sentence is "Pink hate sleep Monday." The third sentence is "Pink hate sleep Monday."
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer:

1.  Pink Monday is here! 😴💖  
2.  Hate Mondays, but I'm ready for a good night's sleep. 😴💤
3.  Pink and comfy, ready for a relaxing Monday. 💖🛌


Let me know if you'd like me to try generating some more! 😊 
<end_of_turn>


# rank

In [174]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate and rank three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer without explanation:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate and rank three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer without explanation:
1. Pink hate sleep Monday
2. Hate pink sleep Monday
3. Sleep pink hate Monday
Explanation:
1. The first sentence is about a person who is not happy with their life and is feeling depressed. They are not sleeping well and are feeling like they are being attacked by pink hate.
2. The second sentence is about a person who is feeling angry and is not sleeping well. They are feeling like they are being attacked by pink hate.
3. The third sentence is about a person who is feeling sad and is not sleeping well. They are feeling like they are being attacked by pink hate.
1. Pink hate sleep Monday
2. Hate pink sleep Monday
3. Sleep pink hate Monday
Explanation:
1. The first sentence is about a person who is not happy with their life and is feeling depressed. They are not sleeping well and are feeling like they are being attac

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate and rank three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer without explanation:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate and rank three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer without explanation:

1.  Monday blues got me feeling pink and hating sleep 😴💔
2.  Pink Monday vibes, but I'm just trying to get through the day without hating sleep 😴💖
3.  Hate Monday, love pink and sleep 😴💖🚫

<end_of_turn>


# rank + 沒有without explanation

In [193]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate and rank three simple sentences for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate and rank three simple sentences for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer:
1. Pink hate sleep Monday
2. Hate pink sleep Monday
3. Sleep hate pink Monday
Explanation:
The first sentence is a simple sentence with all of the words in the prompt. The second sentence is a simple sentence with the words in the opposite order. The third sentence is a simple sentence with the words in the same order as the prompt.
The first sentence is the most likely to be used for an Instagram post because it is the most direct and to the point. The second sentence is less likely to be used because it is more complicated and may not be as clear. The third sentence is the least likely to be used because it is the most complicated and may not be as clear.
The first sentence is the most likely to be used for an Instagram post because it is the most direct and to the point. The second sentence is less likely to be used because it is more complica

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate and rank three simple sentences for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate and rank three simple sentences for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer:

Here are three simple sentences for an Instagram post, incorporating the words "pink," "hate," "sleep," and "Monday":

1. **Pink Monday blues got me hating Mondays.  😴  Wish I could just sleep all day. 😴** (This sentence uses the words in a relatable way, focusing on the negative feelings of Monday.)
2. **Pink skies and a Monday morning.  Hate to admit it, but I'm ready for some sleep. 😴** (This sentence uses the words in a more playful way, highlighting the contrast between the beautiful scenery and the desire for sleep.)
3. **Monday morning, pink and tired.  Hate to say it, but I'm ready for a good night's sleep. 😴** (This sentence is a bit more direct and honest, acknowledging the tiredness associated with Mondays.)

**Ranking:**

1. **Pink Monday blues got me hating Mondays.  😴  Wish I could just


In [170]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday.
 Answer:
1. Pink hate sleep Monday
2. Pink hate sleep Monday
3. Pink hate sleep Monday
Explanation:
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second sentence is "Pink hate sleep Monday." The third sentence is "Pink hate sleep Monday."
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second sentence is "Pink hate sleep Monday." The third sentence is "Pink hate sleep Monday."
The prompt is to generate three simple sentences with emoji for Instagram post with all of the following words: pink, hate, sleep, monday. The first sentence is "Pink hate sleep Monday." The second

In [160]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: blue, hate, sleep, monday.
 Answer:
1. Blue Monday is a day of the week that is often associated with feelings of sadness and depression.
2. Hate Monday is a day that is often associated with feelings of anger and resentment.
3. Sleep Monday is a day that is often associated with feelings of fatigue and exhaustion.<eos>


# normal 好詞有用 +X emoji

In [194]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:
1. Squeeze the last bit of sleep out of your eyes.
2. Tokens of love are the best way to show your affection.
3. Sleep is the best way to recharge your batteries.<eos>


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:

1.  **Squeeze** in some extra sleep tonight, you deserve it! 😴  #special #tokens
2.  These **tokens** are so special, they'll make you want to **sleep** all day! 💤 #squeeze
3.  **Sleep** well, recharge your **tokens** and be ready for a special day! ✨ #tokens #squeeze 


Let me know if you'd like to try a different combination of words! 
<end_of_turn>


# normal 好詞有用 + emoji

In [179]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:
1. Squeeze the tokens into your sleep schedule.
2. Special tokens for special sleep.
3. Sleep tokens for special sleep.<eos>


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = ("Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoji for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:

1.  **Squeeze** in some extra **tokens** before you **sleep** tonight. 😴 
2.  This **special** **tokens** event is going to be **sleep**ing on the couch. 💤
3.  **Squeeze** in some **tokens** for your favorite **sleep** app. 😴 


Let me know if you'd like to try a different combination of words! 😊 
<end_of_turn>


# normal 好詞有用 + emoticons

In [191]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emoticons for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoticons for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:
1. Squeeze the tokens and sleep in the special place.
2. The tokens are special and sleep in the special place.
3. The tokens are special and sleep in the special place.<eos>


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emoticons for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emoticons for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:

1.  **Squeeze** in some extra tokens before you **sleep**! 😴  #sleepwell #tokens
2.  This **special** **squeeze** is the perfect way to get your tokens! 🎁  #specialoffer #tokens
3.  **Sleep** tight and dream of **special** tokens! 💤  #dreambig #tokens 


Let me know if you'd like me to try generating some more! 😊 
<end_of_turn>


# normal 好詞有用 + emotions

In [184]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emotions for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emotions for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:
1. Squeeze the tokens of sleep into your day.
2. Sleep is a special token of love.
3. Squeeze the tokens of sleep into your day.<eos>


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emotions for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emotions for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:

1. **Squeezing** my favorite tokens tight, I can't wait to finally **sleep** on this special night.
2. **Special** memories are made with **tokens** and **sleep**, a perfect combination for a truly unforgettable night.
3. **Squeeze** the moment, cherish the memories, and **sleep** on the special feeling. 


Let me know if you'd like to explore other variations or themes! 😊 
<end_of_turn>


# normal 好詞有用 + emojis

In [192]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emojis for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emojis for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:
1. Squeeze the day away with a special token of love.
2. Sleep tight with a special token of love.
3. Special tokens of love are the best way to squeeze the day away.<eos>


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = (
    "Prompt: Generate three simple sentences with emojis for Instagram post with all of the following words: squeeze, tokens, sleep, special.\n Answer:")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Prompt: Generate three simple sentences with emojis for Instagram post with all of the following words: squeeze, tokens, sleep, special.
 Answer:

1.  **Squeeze** in some **tokens** before you **sleep** 😴.
2.  This **special** **tokens** event is going to be **sleep**ing on the **squeeze**! 🤫
3.  **Sleep** well and dream of **special** **tokens** 💤✨. 


Let me know if you'd like me to try generating some more! 😊 
<end_of_turn>


# 

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

🍎 Nice apple 🍎
😎 Cool 😎
🤯 Amaze 🤯

What's your favorite thing about apples? 🤔

#apple #fruit #healthy #delicious #fall #autumn #food #foodie #instafoodie #healthylifestyle #healthychoices #goodvibes 


**Explanation:**

* **Catchy and concise:** The copy is short, sweet, and uses emojis to grab attention.
* **Highlights key features:** It emphasizes the "nice," "cool," and "amaze" aspects of apples.
* **Engaging question:** It encourages interaction by asking what people's favorite things about apples are.
* **Relevant hashtags:** The hashtags are a mix of general and specific to apples, making it discoverable by a wider audience.


**Tips for using this copy:**

* **Use high-quality images:**  A visually appealing photo of an apple will make your post more engaging.
* **Add a call to


# Grmma2-2b

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly with emoji and all of the following words together: apple, nice, cool, amaze.

Write a sentence for Instagram possibly wit

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "apple, nice, cool, amaze"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice, cool, amaze, cool, nice, apple, nice


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha.

Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha.

Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha.

Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha.

Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED, Cytometry, Inorganic, yr, 躅, praxis, svr, rm, acyj, Baja, knees, Tha.

Write a meme for Instagram with all of the following elements: polite, britann, SUPPORTED,


# Grmma2-2b-it

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a meme for Instagram with all of the following elements: blue, hate, sleep, monday, burger."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))


<bos>Write a meme for Instagram with all of the following elements: blue, hate, sleep, monday, burger.

**Image:** A picture of a burger with a blue bun.

**Caption:** "Me trying to get through Monday with zero sleep and a whole lot of hate.  🍔😩💙 #MondayMotivation #Sleepless #BurgerLove"

**Explanation:**

* **Blue Bun:**  The blue bun is a visual cue for the "hate" and "blue" elements.
* **Burger:**  The burger is the main focus, representing the "Monday motivation" and the desire to enjoy something good.
* **Zero Sleep:**  The caption highlights the lack of sleep, which is a common struggle for many on Mondays.
* **Hate:**  The caption uses "hate" to represent the negative emotions associated with Mondays.
* **Monday Motivation:**  The caption uses "Monday motivation" to acknowledge the struggle but also to suggest that there's a reason to keep going.


This meme plays on the common experience of hating Mondays and the desire to find something


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a meme for Instagram copywriting with all of the following elements: blue, hate, sleep, monday, burger."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a meme for Instagram copywriting with all of the following elements: blue, hate, sleep, monday, burger.

**Image:** A blue, sad-looking burger with a single tear rolling down its bun.

**Caption:**  "Monday morning blues got you feeling like this? 😩🍔  Hate the thought of another week of work?  Sleep is for the weak.  Let's face it, we all need a burger break.  #MondayMotivation #BurgerLove #SleeplessMonday #BlueMonday"

**Explanation:**

* **Blue:**  The blue color is a visual cue for sadness and melancholy, which aligns with the "Monday blues" theme.
* **Hate:**  The caption uses the word "hate" to emphasize the negative feelings associated with Monday.
* **Sleep:**  The caption mentions "sleep is for the weak," which is a humorous and relatable way to express the desire to avoid work.
* **Monday:**  The caption directly references Monday, making it clear that the meme is targeted at this specific day.
* **Burger:**  


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a copy for Instagram possibly with emoji and all of the following words together: blue, hate, sleep, monday, burger."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids
# elements 會拆解詞性
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a copy for Instagram possibly with emoji and all of the following words together: blue, hate, sleep, monday, burger.

#MondayMotivation 🍔💙😴 #HateMondays 

**Here are some ideas for your caption:**

* **Playful:** "Blue skies, hate Mondays, and a burger craving?  🍔💙😴  What's your Monday morning mood?"
* **Relatable:** "Mondays are rough, but at least I have a burger to look forward to! 🍔💙😴 #HateMondays"
* **Motivational:** "Blue skies and a fresh start!  Let's conquer this Monday with a burger in hand. 💪🍔💙😴 #MondayMotivation"


**Choose the caption that best fits your brand and audience!** 
<end_of_turn>


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a humor memetic post for Instagram with the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a humor memetic post for Instagram with the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles,  

**Note:**  This is a humorous take on the world of online marketing and social media. 

**Image:** A photo of someone looking confused and holding a string with a single, very small, "0" on it.

**Caption:**

"Just trying to figure out the latest marketing trend. Apparently, it's all about 'immersive experiences' and 'brand partnerships' with 'linear channels' and 'badass' fonts.  But I'm still stuck on this:  **Sight, thrower, string, password, zeros, brand, pair, raffles, news, channel, linear, font, commenting, inspiration, great.**  What am I missing? 🤔  Anyone got a clue?  #marketingconfusion #cantkeepup #givemeabreak"


**Explanation:**

* **Humorous Tone:** The meme uses a relatable scenario of confusion and frustration with the ever-changing world of online

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a memetic post for Instagram with the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles, Apple, Banana, Bread, Chair, Cloud, Dog, Elephant, Fish, Giraffe, Grass, Hat, House, Ice cream, Jellyfish, Kite, Lemon, Lion, Monkey, Nest, Octopus, Parrot, Quilt, Rabbit, Snake, Tiger, Umbrella, Vase, Watermelon, Xylophone, Yak, Zebra."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a memetic post for Instagram with the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles, Apple, Banana, Bread, Chair, Cloud, Dog, Elephant, Fish, Giraffe, Grass, Hat, House, Ice cream, Jellyfish, Kite, Lemon, Lion, Monkey, Nest, Octopus, Parrot, Quilt, Rabbit, Snake, Tiger, Umbrella, Vase, Watermelon, Xylophone, Yak, Zebra.

**Meme Caption:**

"I'm so inspired by the sheer **sight** of this **password** that I'm going to **throw** my **string** at it and **immerse** myself in the **linear** world of **brand** and **channel**."

**Explanation:**

* **Inspiration:** The meme uses the word "sight" to evoke a sense of awe and wonder, implying that the password is so impressive that it inspires the user.
* **Password:** The password is a key element of the meme, as it is the object of the user's inspiration. The meme uses the word "password" to create a sense of mys

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a humor memetic post for Instagram possibly with all the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles, Apple, Banana, Bread, Chair, Cloud, Dog, Elephant, Fish, Giraffe, Grass, Hat, House, Ice cream, Jellyfish, Kite, Lemon, Lion, Monkey, Nest, Octopus, Parrot, Quilt, Rabbit, Snake, Tiger, Umbrella, Vase, Watermelon, Xylophone, Yak, Zebra."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a humor memetic post for Instagram possibly with all the following elements: inspiration, Great , password, Sight, thrower  , string ,Immerse , font, commenting , news , zeros, brand, linear,channel, Badass , pair, raffles, Apple, Banana, Bread, Chair, Cloud, Dog, Elephant, Fish, Giraffe, Grass, Hat, House, Ice cream, Jellyfish, Kite, Lemon, Lion, Monkey, Nest, Octopus, Parrot, Quilt, Rabbit, Snake, Tiger, Umbrella, Vase, Watermelon, Xylophone, Yak, Zebra.

**Caption:**

"I'm so immersed in this new password generator that I forgot to check the news.  Apparently, Apple is launching a new line of bread-shaped iPhones.  I'm thinking of entering the raffle for a pair of banana-flavored headphones.  Maybe I'll even get a chair to sit on while I listen to the music.  What's your password?  #passwordgenerator #apple #banana #raffles #badass #immersive"

**Image:**

A screenshot of a password generator with a long, complex password.  The background is a blurry image of a person sit

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Write a humor memetic post for Instagram with the following elements: SAX, Gareth, Fernseh, arbeitet, konden, tanım,ową, sile, Advent, rapporti, Allowed, ΟΣ, 銳, istive, Kill."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Write a humor memetic post for Instagram with the following elements: SAX, Gareth, Fernseh, arbeitet, konden, tanım,ową, sile, Advent, rapporti, Allowed, ΟΣ, 銳, istive, Kill.

This is a very specific and niche meme, so it's important to understand the context before posting. 

**Context:**

* **Gareth:** A popular character in a specific online community known for his humor and often nonsensical statements.
* **Fernseh:** A German word for "television."
* **Advent:** A Christian season of preparation for Christmas.
* **Rapports:** A term used in the community to refer to gossip or rumors.
* **Allowed:** A common phrase used in the community to indicate something is acceptable or permissible.
* **OS:** A common abbreviation for "Operating System" in the community.
* **Istive:** A slang term used in the community to describe something that is very strong or powerful.
* **Kill:** A common slang term used in the community to express excitement or approval.

**Meme Format:**

* **Image

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gemma.to(device)
input_text = "Give me 64 random words in this format: word1, word2, word3"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = gemma.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Give me 64 random words in this format: word1, word2, word3, ... word64

Here's a list of words to choose from:

* Apple
* Banana
* Bread
* Chair
* Cloud
* Dog
* Elephant
* Fish
* Giraffe
* Grass
* Hat
* House
* Ice cream
* Jellyfish
* Kite
* Lion
* Moon
* Orange
* Penguin
* Pig
* Rainbow
* River
* Rose
* Sun
* Table
* Tree
* Umbrella
* Violin
* Water
* Whale
* Wind
* Wood

**Please note:** 

* You can use any combination of words from the list.
* The words should be in a random order.
* You can repeat words if you want.


Here's an example of how the output should look:

Apple, Banana, Bread, Chair, Cloud, Dog, Elephant, Fish, Giraffe, Grass, Hat, House, Ice cream, Jellyfish
